In [1]:
import numpy as np
import re
from sklearn.tree import DecisionTreeRegressor

In [2]:
dataFn = "CASP_f991_dep"
dataTrain = np.genfromtxt(dataFn + "_train.csv", delimiter = ",")
dataTest = np.genfromtxt(dataFn + "_test.csv", delimiter = ",")
fkMatch = re.search(r"f([0-9]+)", dataFn)
if fkMatch:
    dF = int(fkMatch.group(1))
else:
    dF = 0  
dT = dataTrain.shape[1] - dF - 1

In [3]:
nTrain = dataTrain.shape[0]
nTest = dataTest.shape[0]
nOOS = int(min(nTrain / 2, 5e3))
d = dataTrain.shape[1] - 1

np.random.seed(123)
idxOOS = np.random.choice(nTrain, nOOS, replace = False)
XTrain = np.delete(dataTrain[:, :d], idxOOS, axis = 0)
XOOS = dataTrain[idxOOS, :d]
XTest = dataTest[:, :d]
yTrain = np.delete(dataTrain[:, d], idxOOS)
yOOS = dataTrain[idxOOS, d]
yTest = dataTest[:, d]

nTrain = nTrain - nOOS

In [4]:
depths = range(3, 20)
OOSScores = []
for i in depths:
    regTree = DecisionTreeRegressor(max_depth = i, max_features = min(100, XTrain.shape[1]), random_state = 123)
    regTree.fit(XTrain, yTrain)
    yOOSPred = regTree.predict(XOOS)
    OOSScores.append(np.sqrt(np.square(yOOSPred - yOOS).sum() / nOOS))
    if np.argmin(OOSScores) == len(OOSScores) - 1:
        regTreeBest = regTree
    else:
        break

In [5]:
yTestPred = regTreeBest.predict(XTest)
np.square(yTest - yTestPred).sum() / np.var(yTest) / XTest.shape[0]

0.8438246020212803

In [6]:
OOSScores

[5.644093846699301, 5.621423875410234, 5.640117222796912]

In [7]:
(regTreeBest.feature_importances_ > 0).sum()

10

In [8]:
np.sum((regTreeBest.feature_importances_ > 0).nonzero()[0] >= dT) / (regTreeBest.feature_importances_ > 0).sum()

0.5